<a href="https://colab.research.google.com/github/aninsung/Machine-Learning-Programming/blob/main/4%EC%A3%BC%EC%B0%A8_mhealth_cnn%2Clstm%2Ccnn%2Blstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## (오늘 실습과제)

Mhealth dataset으로 인간행동 분류하기

일단 데이터 읽어서 subject 번호가 컬럼면에 포함된 통합 data frame 만들고 CNN, LSTM, CNN-LSTM 해보기.

1. LLM(sLLM)을 제외한 어떤 모델을 써도 가능. 단 라이브러리 import 불가. 모델의 풀 소스가 있어야 함.

 2. 10명을 1+2, 3+4, 4+5,.... 5개그룹으로 나눠 2명씩 테스트 데이터로 하면서 cross validation.

 3. 데이터 증대는 안됨.

4. 오직 모델로만 성능 향상

5. 상위 30%씩 차등.

6. 성적에 반영

7. 11월15일까지. Github게시

교차 평가로 평균 F1 기준.

8. 본인이 완벽하게 소스 및 모델을 설명할 수 있어야 함. GPT를 쓰던 뭘 쓰던 그건 상관안함. 재현성이 보장되어야 함.
--------------------------------------------------------------------------
--------------------------------------------------------------------------

## 데이터설명
📌 Dataset Summary

Subjects: 10명 (다양한 프로필의 자원자)

Activities: 12개

Sensors: 3개 (가슴, 오른쪽 손목, 왼쪽 발목 → elastic strap으로 고정)

측정 정보:

Acceleration (가속도)

Gyroscope (회전율)

Magnetometer (자기장 방향)

Chest sensor: 2-lead ECG 추가 (활동 인식에는 사용 안 함, 향후 연구용)

Sampling Rate: 50 Hz

환경: Out-of-lab, 제약 없이 자유롭게 활동 수행 (단, 최선을 다해 실행)

📌 Activity Set (12 classes)

정적 활동: Standing still, Sitting, Lying down

일상/이동 활동: Walking, Climbing stairs, Cycling

운동/역동적 활동: Jogging, Running, Jump front & back

반복 동작: Waist bends, Arm elevation, Knees bending

📌 특징

ECG 포함: 운동 중 심전도 기록 → 부정맥, 심박수 모니터링, 운동 효과 분석 가능

다양성 확보:

동작 강도 차이 (예: Sitting vs. Running)

속도 차이 (예: Walking vs. Jogging)

다른 신체 부위 동작 (예: Arm elevation vs. Knees bending)

실생활 반영: 실제 환경에서 수행되어 일반화 성능 높음

In [71]:
import pandas as pd
import glob
import numpy as np

# 1. 파일 리스트
files = sorted(glob.glob("/content/drive/MyDrive/Colab Notebooks/mhealth+dataset/MHEALTHDATASET/mHealth_subject*.log"))

# 2. 컬럼 정의
columns = [
    "chest_acc_x","chest_acc_y","chest_acc_z",
    "ecg_1","ecg_2",
    "chest_gyro_x","chest_gyro_y","chest_gyro_z",
    "chest_mag_x","chest_mag_y","chest_mag_z",
    "ankle_acc_x","ankle_acc_y","ankle_acc_z",
    "ankle_gyro_x","ankle_gyro_y","ankle_gyro_z",
    "ankle_mag_x","ankle_mag_y","ankle_mag_z",
    "wrist_acc_x","wrist_acc_y","wrist_acc_z",
    "label"
]

# 3. 각 파일 읽어서 subject_id 추가
dfs = []
for i, file in enumerate(files, start=1):
    df = pd.read_csv(file, sep="\s+", header=None)
    df.columns = columns
    df["subject_id"] = i
    dfs.append(df)

# 4. 통합 DataFrame
data = pd.concat(dfs, ignore_index=True)


<>:24: SyntaxWarning: invalid escape sequence '\s'
<>:24: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4155912323.py:24: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file, sep="\s+", header=None)


In [72]:
def create_windows(df, frame_size=100, hop_size=50):
    X, y = [], []
    data_values = df.drop(["label", "subject_id"], axis=1).values
    labels = df["label"].values
    for i in range(0, len(data_values)-frame_size, hop_size):
        X.append(data_values[i:i+frame_size])
        y.append(labels[i+frame_size-1])  # 마지막 시점 label
    return np.array(X), np.array(y)


In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout

def build_cnn(timesteps, n_features, n_classes):
    model = Sequential([
        Conv1D(64, kernel_size=3, activation='relu', input_shape=(timesteps, n_features)),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Conv1D(128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.3),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(n_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [77]:
from sklearn.metrics import f1_score
import numpy as np

# 결과 저장용 리스트
acc_list = []
loss_list = []
f1_list = []

subjects = np.arange(1,11)
folds = [(subjects[i], subjects[i+1]) for i in range(0,10,2)]

for fold_idx, test_subjects in enumerate(folds, start=1):
    # Train/Test 분리
    test_data = data[data["subject_id"].isin(test_subjects)]
    train_data = data[~data["subject_id"].isin(test_subjects)]

    # 윈도우 생성
    X_train, y_train = create_windows(train_data, frame_size=timesteps, hop_size=50)
    X_test, y_test = create_windows(test_data, frame_size=timesteps, hop_size=50)

    print(f"\n=== Fold {fold_idx}: Test subjects {test_subjects} ===")
    print("Samples:", X_train.shape[0], "(train)", X_test.shape[0], "(test)")

    # 모델 선택
    # CNN, LSTM, CNN-LSTM 중 선택
    model = build_cnn_lstm(timesteps, n_features, n_classes)
    # 학습
    model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

    # 평가
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # F1-score 계산 (다중 클래스는 'macro' 추천)
    f1 = f1_score(y_test, y_pred_classes, average='macro')

    print(f"Test Accuracy: {acc:.4f}, Loss: {loss:.4f}, F1-score: {f1:.4f}")

    # 리스트에 저장
    acc_list.append(acc)
    loss_list.append(loss)
    f1_list.append(f1)

# Cross-validation 평균
print("\n=== Cross-validation Average ===")
print(f"Average Accuracy: {np.mean(acc_list):.4f}")
print(f"Average Loss: {np.mean(loss_list):.4f}")
print(f"Average F1-score: {np.mean(f1_list):.4f}")


=== Fold 1: Test subjects (np.int64(1), np.int64(2)) ===
Samples: 19122 (train) 5190 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - accuracy: 0.7048 - loss: 1.1067
Epoch 2/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 18s 60ms/step - accuracy: 0.7925 - loss: 0.4755
Epoch 3/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8238 - loss: 0.3838
Epoch 4/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8474 - loss: 0.3348
Epoch 5/5
299/299 ━━━━━━━━━━━━━━━━━━━━ 18s 60ms/step - accuracy: 0.8510 - loss: 0.3190
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Test Accuracy: 0.8291, Loss: 0.5507, F1-score: 0.6098

=== Fold 2: Test subjects (np.int64(3), np.int64(4)) ===
Samples: 19260 (train) 5052 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 20s 60ms/step - accuracy: 0.7165 - loss: 1.0601
Epoch 2/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.7923 - loss: 0.4691
Epoch 3/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8254 - loss: 0.3754
Epoch 4/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8411 - loss: 0.3434
Epoch 5/5
301/301 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8512 - loss: 0.3185
158/158 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
Test Accuracy: 0.8040, Loss: 0.6626, F1-score: 0.5607

=== Fold 3: Test subjects (np.int64(5), np.int64(6)) ===
Samples: 19583 (train) 4729 (test)


...

Epoch 1/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 20s 60ms/step - accuracy: 0.7226 - loss: 1.0708
Epoch 2/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 18s 60ms/step - accuracy: 0.7934 - loss: 0.4779
Epoch 3/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8298 - loss: 0.3826
Epoch 4/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 19s 63ms/step - accuracy: 0.8417 - loss: 0.3462
Epoch 5/5
306/306 ━━━━━━━━━━━━━━━━━━━━ 18s 60ms/step - accuracy: 0.8551 - loss: 0.3138
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Test Accuracy: 0.7928, Loss: 0.5461, F1-score: 0.6043

=== Fold 4: Test subjects (np.int64(7), np.int64(8)) ===
Samples: 20258 (train) 4054 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 21s 61ms/step - accuracy: 0.7237 - loss: 1.0404
Epoch 2/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 19s 61ms/step - accuracy: 0.7949 - loss: 0.4522
Epoch 3/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.8223 - loss: 0.3852
Epoch 4/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.8362 - loss: 0.3411
Epoch 5/5
317/317 ━━━━━━━━━━━━━━━━━━━━ 19s 60ms/step - accuracy: 0.8506 - loss: 0.3080
127/127 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Test Accuracy: 0.8029, Loss: 0.5372, F1-score: 0.6729

=== Fold 5: Test subjects (np.int64(9), np.int64(10)) ===
Samples: 19030 (train) 5282 (test)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 19s 59ms/step - accuracy: 0.6989 - loss: 1.1208
Epoch 2/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.7844 - loss: 0.4896
Epoch 3/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8195 - loss: 0.4010
Epoch 4/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8353 - loss: 0.3541
Epoch 5/5
298/298 ━━━━━━━━━━━━━━━━━━━━ 18s 59ms/step - accuracy: 0.8488 - loss: 0.3304
166/166 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Test Accuracy: 0.8200, Loss: 0.4372, F1-score: 0.6632

=== Cross-validation Average ===
Average Accuracy: 0.8098
Average Loss: 0.5468
Average F1-score: 0.6222
